<h1> Classification with a Neural Network using PyTorch</h1>

<h2>1. Imports and load data</h2>

In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class DataProcessor:
    def __init__(self, input_path: str, file_names: List[str]) -> None:
        """
        Initializes the DataProcessor with the given input path and file names.

        Args:
            input_path (str): The directory path where the files are stored.
            file_names (List[str]): List of file names to be read.

        Returns:
            None
        """
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self) -> Dict[str, pd.DataFrame]:
        """
        Reads the files specified in the file names and stores them in a dictionary.

        Returns:
            Dict[str, pd.DataFrame]: A dictionary where keys are file names and values are DataFrames.
        """
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self) -> None:
        """
        Prints the shape of each loaded DataFrame.

        Returns:
            None
        """
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self) -> pd.Series:
        """
        Renames the columns in the data['target'] and creates the wanted target DataFrame by extracting the column 'Valve_Condition'.

        Returns:
            pd.Series: A pandas Series containing the 'Valve_Condition' column from the target DataFrame.
        """
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                          'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                          'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        return self.valve_condition

def process_data() -> (Dict[str, pd.DataFrame], pd.Series): # type: ignore
    """
    Processes the data by reading the files and extracting the target DataFrame.

    Returns:
        Tuple[Dict[str, pd.DataFrame], pd.Series]: A tuple containing the data dictionary and the valve condition Series.
    """
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


<h2>2. Create input and target data </h2>

We use the six sensors which we identified as relevant during data exploration: 'eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3' 

In [3]:
df_list = ['eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3']
input_df = pd.concat([data[i] for i in df_list], axis = 1)
input_df

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2409.6,...,2.336,2.391,2.375,2.297,2.328,2.383,2.328,2.250,2.250,2.211
1,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,...,2.297,2.266,2.266,2.219,2.211,2.266,2.273,2.211,2.195,2.219
2,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2395.8,...,2.359,2.391,2.391,2.375,2.375,2.375,2.305,2.305,2.320,2.266
3,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2382.8,2382.8,...,2.117,2.219,2.281,2.227,2.164,2.164,2.219,2.250,2.273,2.273
4,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2373.0,...,2.141,2.172,2.187,2.227,2.219,2.211,2.242,2.219,2.227,2.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,...,2.328,2.305,2.328,2.359,2.375,2.281,2.242,2.250,2.266,2.273
2201,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,...,2.273,2.383,2.359,2.297,2.297,2.336,2.406,2.461,2.461,2.406
2202,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.227,2.242,2.219,2.211,2.273,2.273,2.250,2.219,2.219,2.250
2203,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.328,2.328,2.328,2.281,2.266,2.305,2.281,2.250,2.242,2.281


Standardise the input and target data

In [4]:
# Standardise the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

# Standatdise the input
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_df)

<h2>3. Create the model, train it & make predictions </h2>

<ul>
<li>We create a neural network with 1 input layer, 2 hidden layers and 1 output layer --> 4 layers total
<li>We use the stochastic gradient descent with a middle-sized batch size of 32 since we dont have a very big data set
<li>We use CrossEntropyLoss() for calculcating the loss. It uses the softmax function. Because of that, we don't have to use the softmax function in our output layer for classification by using the probabilities
</ul>

In [5]:
states = [27, 6728, 49122]
accs = []

In [6]:
for RANDOM_STATE in states:
    # split into train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state=RANDOM_STATE, stratify=y_encoded)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp)

    # create tensors for pytorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # create datasets for train, validation, and test
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    val_data = TensorDataset(X_val_tensor, y_val_tensor)
    test_data = TensorDataset(X_test_tensor, y_test_tensor)

    # create DataLoader for train, validation, and test
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    class Network(nn.Module):
        def __init__(self, input_size: int, output_size: int, hidden_size: int = 128, num_layers: int = 4, dropout: float = 0.2) -> None:
            """
            Initializes a neural network model with specified parameters.

            The network consists of an input layer, hidden layers, and an output layer, 
            with ReLU activations, dropout regularization, and a specified number of layers.

            Parameters:
            -----------
            input_size : int
                The number of input features.
            
            output_size : int
                The number of output units (e.g., for classification or regression).
            
            hidden_size : int, optional, default=128
                The number of units in the hidden layers.
            
            num_layers : int, optional, default=4
                The number of hidden layers (currently fixed at 4 for simplicity).
            
            dropout : float, optional, default=0.2
                The dropout rate to apply after each hidden layer.
            """
            super(Network, self).__init__()
            
            # Initialize layers
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.fc3 = nn.Linear(hidden_size, hidden_size)
            self.fc4 = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)
            
            # Initialize activation function
            self.relu = nn.ReLU()
        
        def forward(self, x: torch.Tensor) -> torch.Tensor:
            """
            Defines the forward pass of the network.

            Parameters:
            -----------
            x : torch.Tensor
                The input tensor to the model.

            Returns:
            --------
            torch.Tensor
                The output tensor after passing through the network layers and activation functions.
            """
            x = self.relu(self.fc1(x))  
            x = self.dropout(x)        
            x = self.relu(self.fc2(x)) 
            x = self.dropout(x)         
            x = self.relu(self.fc3(x))  
            x = self.dropout(x)        
            x = self.fc4(x)             
            return x

        def score_function(engine):
            val_loss = engine.state.metrics['nll']
            return -val_loss


    # defining the model
    model = Network(input_size=X_train.shape[1], output_size=4)  # output size because we have 4 classes 
    
    # calculating the loss and optimizer
    criterion = nn.CrossEntropyLoss()  # Loss function for classification
    optimizer = optim.Adam(model.parameters(), lr=0.002)
    
    # training the model, using 100 epochs for each random state
    num_epochs = 100
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # using the scheduler to adapt the learning rate dynamically during training

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0 
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
             # forward Pass
            outputs = model(inputs)
            
            # calculate the loss with the loss function
            loss = criterion(outputs, targets)
            
            # backward pass and optimzing weights
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        scheduler.step()
        
        # evaluate on validation set after each epoch
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                
                preds = torch.argmax(outputs, axis=1)
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(targets.cpu().numpy())
        
    # store preds and targets for evaluation on the test set
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            preds = torch.argmax(outputs, axis=1)   # the max. probability defines the predicted class
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    # print the classificiation report
    print(f"Classification Report for Random State {RANDOM_STATE}:")
    print(classification_report(all_targets, all_preds, zero_division=0.0))

    # calcualte accuracy and append it to the list to calculate mean and std later
    accuracy = accuracy_score(all_targets, all_preds)
    accs.append(accuracy)

# Calculate mean and std of accuracy scores
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")

Classification Report for Random State 27:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        36
           3       1.00      1.00      1.00       113

    accuracy                           1.00       221
   macro avg       1.00      1.00      1.00       221
weighted avg       1.00      1.00      1.00       221

Classification Report for Random State 6728:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        36
           1       0.97      1.00      0.99        36
           2       0.97      0.97      0.97        36
           3       1.00      1.00      1.00       113

    accuracy                           0.99       221
   macro avg       0.99      0.99      0.99       221
weighted avg       0.99      0.99      0.99       221

Classification Report for Random State 491